In [ ]:
import skimage
from PIL import Image
import glob
import os
import natsort
import numpy as np
from skimage.transform import resize
import random
from tqdm import trange, tqdm
import cv2
import multiprocessing as mp
import time
import sklearn.metrics as metrics
from pyquaternion import Quaternion
import math
import itertools
import sys
%matplotlib inline
import matplotlib.pyplot as plt
from collections import OrderedDict
import copy
import shutil

from torchvision import transforms
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from my_networks import scr_net, CreateDiscriminator

from skimage import data
from skimage import exposure
from skimage.exposure import match_histograms
import scipy.misc


In [ ]:
# the location of training rendered images
main_path = '/scratch/zq415/grammar_cor/pose/pose_estimate/icra_data'
# the location to save the rendered images after histogram matching
root_save_path = '../train_render_matched'


In [ ]:
render_img_paths = natsort.natsorted(glob.glob(main_path+'/rendered/**/*.png', recursive=True))


In [ ]:
# pick 8 real camera images to provide the target histogram information
ref_path1 = '/scratch/zq415/grammar_cor/pose/pose_estimate/icra_data/train_real_imgs/2'
ref_path2 = '/scratch/zq415/grammar_cor/pose/pose_estimate/icra_data/train_real_imgs/5'

reference_1 = np.asarray(Image.open(os.path.join(ref_path1, '01590.png')).convert('RGB'))
reference_2 = np.asarray(Image.open(os.path.join(ref_path1, '03358.png')).convert('RGB'))
reference_3 = np.asarray(Image.open(os.path.join(ref_path1, '08140.png')).convert('RGB'))
reference_4 = np.asarray(Image.open(os.path.join(ref_path1, '16748.png')).convert('RGB'))

reference_5 = np.asarray(Image.open(os.path.join(ref_path2, '00934.png')).convert('RGB'))
reference_6 = np.asarray(Image.open(os.path.join(ref_path2, '06168.png')).convert('RGB'))
reference_7 = np.asarray(Image.open(os.path.join(ref_path2, '08578.png')).convert('RGB'))
reference_8 = np.asarray(Image.open(os.path.join(ref_path2, '39896.png')).convert('RGB'))



In [ ]:
matched_num = 0
for i in range(len(render_img_paths)):
    matched_num += 1
    print(matched_num)
    sub_folder_name = render_img_paths[i].split('/')[-2]
    img_name = render_img_paths[i].split('/')[-1]
    
    img_save_path = os.path.join(root_save_path, sub_folder_name)
    if not os.path.exists(img_save_path):
        os.makedirs(img_save_path)
    img_save_name = os.path.join(img_save_path, img_name)
    if os.path.exists(img_save_name):
        continue
    
    render_img = np.asarray(Image.open(render_img_paths[i]).convert('RGB'))
    
    matched_1 = match_histograms(render_img, reference_1, multichannel=True)
    matched_2 = match_histograms(render_img, reference_2, multichannel=True)
    matched_3 = match_histograms(render_img, reference_3, multichannel=True)
    matched_4 = match_histograms(render_img, reference_4, multichannel=True)
    
    matched_5 = match_histograms(render_img, reference_5, multichannel=True)
    matched_6 = match_histograms(render_img, reference_6, multichannel=True)
    matched_7 = match_histograms(render_img, reference_7, multichannel=True)
    matched_8 = match_histograms(render_img, reference_8, multichannel=True)
    
    # take the average
    matched = np.array(matched_1/8.0+matched_2/8.0+matched_3/8.0+matched_4/8.0+
                       matched_5/8.0+matched_6/8.0+matched_7/8.0+matched_8/8.0, dtype=np.uint8)
    
    
    Image.fromarray(matched).save(img_save_name)
    